https://github.com/vivek081166/raw-audio-deep-learning/blob/master/constants.py

In [3]:
import pickle
from glob import iglob
import numpy as np
import librosa
from shutil import rmtree
from constants import *
# from constants import DATA_AUDIO_DIR

DATA_AUDIO_DIR = './test_train_data'
list_dir = os.listdir(DATA_AUDIO_DIR)
list_dir.sort()
# TARGET_SR = 16000


def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


# def del_folder(path):
#     try:
#         rmtree(path)
#     except:
#         pass


# del_folder(OUTPUT_DIR_TRAIN)
# del_folder(OUTPUT_DIR_TEST)
# mkdir_p(OUTPUT_DIR_TRAIN)
# mkdir_p(OUTPUT_DIR_TEST)

order = 0
class_ids = {list_dir[i]: i for i in range(len(list_dir))}


def extract_class_id(wav_filename):
#     print(class_ids.get(str(wav_filename[18:-6])))
    return class_ids.get(wav_filename[18:-6])


def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio


def convert_data():
    DATA_AUDIO_DIR = './test_train_data'
    list_dir = os.listdir(DATA_AUDIO_DIR)
    list_dir.sort()
#     TARGET_SR = 16000
#     print(list_dir)
#     AUDIO_LENGTH = 200000
    for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_DIR, '**/**.wav'), recursive=True)):
#         print(i, wav_filename)
        class_id = extract_class_id(wav_filename)
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            print('PAD New length =', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
#             audio_buf = audio_buf[10000:AUDIO_LENGTH+10000]
            audio_buf = audio_buf[0:AUDIO_LENGTH]
            print('CUT New length =', len(audio_buf))

        output_folder = OUTPUT_DIR_TRAIN
#         print("wav_filename_end : ", wav_filename[-5:])
        if wav_filename[-5:] == '5.wav':
            output_folder = OUTPUT_DIR_TEST

        output_filename = os.path.join(output_folder, str(wav_filename[18:-4]).replace('/', '_') + '.pkl')
#         TARGET_SR = 16000
        out = {'class_id': class_id,
               'audio': audio_buf,
               'sr': TARGET_SR}
        print("-----------------")
        print(out)
        with open(output_filename, 'wb') as w:
            pickle.dump(out, w)


if __name__ == '__main__':
    convert_data()

0 ./test_train_data/2017019770032_kimdayeong/5.wav 568662 0.0 1.0
CUT New length = 200000
-----------------
{'class_id': 65, 'audio': array([[-0.16159828],
       [-0.13780172],
       [-0.14970757],
       ...,
       [-0.11018832],
       [-0.23004085],
       [-0.37413192]], dtype=float32), 'sr': 16000}
1 ./test_train_data/2017019770032_kimdayeong/1.wav 582315 0.0 0.9999999
CUT New length = 200000
-----------------
{'class_id': 65, 'audio': array([[-0.22189948],
       [-0.21613286],
       [-0.20964813],
       ...,
       [ 0.20203884],
       [ 0.27829698],
       [ 0.294002  ]], dtype=float32), 'sr': 16000}
2 ./test_train_data/2017019770032_kimdayeong/3.wav 548864 0.0 1.0000001
CUT New length = 200000
-----------------
{'class_id': 65, 'audio': array([[-0.06339043],
       [-0.01367757],
       [-0.01256262],
       ...,
       [-1.1160156 ],
       [-1.2334596 ],
       [-1.3049961 ]], dtype=float32), 'sr': 16000}
3 ./test_train_data/2017019770032_kimdayeong/2.wav 526336 0.0 1.

27 ./test_train_data/2017019880039_seojuyeon/3.wav 479574 0.0 0.9999999
CUT New length = 200000
-----------------
{'class_id': 112, 'audio': array([[ 4.1584794e-05],
       [-5.2006869e-03],
       [-1.4371445e-02],
       ...,
       [-9.4218230e-01],
       [-7.8286964e-01],
       [-6.9218183e-01]], dtype=float32), 'sr': 16000}
28 ./test_train_data/2017019880039_seojuyeon/2.wav 464896 -0.0 1.0
CUT New length = 200000
-----------------
{'class_id': 112, 'audio': array([[-0.24811508],
       [-0.54989696],
       [-0.08667471],
       ...,
       [ 0.12825914],
       [ 0.54766613],
       [ 0.98902243]], dtype=float32), 'sr': 16000}
29 ./test_train_data/2017019880039_seojuyeon/4.wav 467627 0.0 1.0000001
CUT New length = 200000
-----------------
{'class_id': 112, 'audio': array([[0.03783536],
       [0.03120072],
       [0.02192613],
       ...,
       [0.0358238 ],
       [0.04302957],
       [0.02128345]], dtype=float32), 'sr': 16000}
30 ./test_train_data/2017019770023_kimsoyung/5.w

54 ./test_train_data/2017019880034_leesujung/4.wav 510096 -0.0 0.99999994
CUT New length = 200000
-----------------
{'class_id': 107, 'audio': array([[-0.02177776],
       [ 0.04886897],
       [-0.05196038],
       ...,
       [-0.32177803],
       [-0.32779232],
       [-0.3216993 ]], dtype=float32), 'sr': 16000}
55 ./test_train_data/2017019880032_seoyeonjoo/5.wav 430080 -0.0 1.0000001
CUT New length = 200000
-----------------
{'class_id': 105, 'audio': array([[0.00358986],
       [0.00230872],
       [0.00852787],
       ...,
       [1.9262273 ],
       [2.0084443 ],
       [1.9830025 ]], dtype=float32), 'sr': 16000}
56 ./test_train_data/2017019880032_seoyeonjoo/1.wav 476160 -0.0 0.99999994
CUT New length = 200000
-----------------
{'class_id': 105, 'audio': array([[-0.00219186],
       [ 0.00347928],
       [-0.00162421],
       ...,
       [-0.37398562],
       [-0.01996159],
       [ 0.39359793]], dtype=float32), 'sr': 16000}
57 ./test_train_data/2017019880032_seoyeonjoo/3.wav 46

KeyboardInterrupt: 

In [258]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow import convert_to_tensor
import numpy as np
import pickle
import os
from glob import glob

# AUDIO_LENGTH = 200000
OUTPUT_DIR = './output'
OUTPUT_DIR_TRAIN = os.path.join(OUTPUT_DIR, 'train')
OUTPUT_DIR_TEST = os.path.join(OUTPUT_DIR, 'test')

DATA_AUDIO_DIR = './test_train_data'
list_dir = os.listdir(DATA_AUDIO_DIR)
list_dir.sort()
# TARGET_SR = 16000

class_ids = {list_dir[i]: i for i in range(len(list_dir))}
def m5(num_classes):
    print("!!!!!!!!", num_classes)
    print('Using Model M5')
    m = Sequential()
    m.add(Conv1D(128,
                 input_shape=[AUDIO_LENGTH, 1],
                 kernel_size=80,
                 strides=4,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(128,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(256,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Conv1D(512,
                 kernel_size=3,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 kernel_regularizer=regularizers.l2(l=0.0001)))
    m.add(BatchNormalization())
    m.add(Activation('relu'))
    m.add(MaxPooling1D(pool_size=4, strides=None))
    m.add(Lambda(lambda x: K.mean(x, axis=1)))  # Same as GAP for 1D Conv Layer
    m.add(Dense(num_classes, activation='softmax'))
    return m


def get_data(file_list):
    def load_into(_filename, _x, _y):
#         print("**** : ", _filename)
#         print("~`~` : ", _filename[15:-6])
        
        with open(_filename, 'rb') as f:
            audio_element = pickle.load(f)
            _x.append(audio_element['audio'])
            _y.append(int(audio_element['class_id']))

    x, y = [], []
    for filename in file_list:
#         print("Predict file name : ", filename)
        load_into(filename, x, y)
    return np.array(x), np.array(y)
#     return x, y




num_classes = len(list_dir)
print(num_classes)
model = m5(num_classes=len(list_dir))

if model is None:
    exit('Something went wrong!!')

model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
print(model.summary())

train_files = glob(os.path.join(OUTPUT_DIR_TRAIN, '**.pkl'))
x_tr, y_tr = get_data(train_files)
# print(len(x_tr))
# print(y_tr)
y_tr = to_categorical(y_tr, num_classes=num_classes)

test_files = glob(os.path.join(OUTPUT_DIR_TEST, '**.pkl'))
x_te, y_te = get_data(test_files)
y_te = to_categorical(y_te, num_classes=num_classes)

print('x_tr.shape =', x_tr.shape)
print('y_tr.shape =', y_tr.shape)
print('x_te.shape =', x_te.shape)
print('y_te.shape =', y_te.shape)
print(type(x_te))

# if the accuracy does not increase over 10 epochs, reduce the learning rate by half.
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.0001, verbose=1)
batch_size = 128
model.fit(x=x_tr, y=y_tr, batch_size=320, epochs=150, verbose=2, shuffle=True, validation_data=(x_te, y_te), callbacks=[reduce_lr])
    
# epoch 500 audio_length 10000 min_lr 0.0001 sr 8000 batch_size 16 : val_acc 0.73
# epoch 500 audio_length 10000 min_lr 0.00005 sr 8000 batch_size 16 : val_acc 0.77(0.83)
# epoch 50 audio_length 20000 min_lr 0.00005 sr 8000 batch_size 16 : val_acc 0.90
# epoch 100 audio_length 20000 min_lr 0.00005 sr 8000 batch_size 16 : val_acc 0.93
# epoch 100 audio_length 20000 min_lr 0.00005 sr 8000 batch_size 32 : val_acc 0.89(0.90) predict 0.75
# epoch 150 audio_length 20000 min_lr 0.00005 sr 8000 batch_size 16 : val_acc 0.91 predict 0.75

118
!!!!!!!! 118
Using Model M5
Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_240 (Conv1D)          (None, 2500, 128)         10368     
_________________________________________________________________
batch_normalization_240 (Bat (None, 2500, 128)         512       
_________________________________________________________________
activation_240 (Activation)  (None, 2500, 128)         0         
_________________________________________________________________
max_pooling1d_240 (MaxPoolin (None, 625, 128)          0         
_________________________________________________________________
conv1d_241 (Conv1D)          (None, 625, 128)          49280     
_________________________________________________________________
batch_normalization_241 (Bat (None, 625, 128)          512       
_________________________________________________________________
activation_241 (Activ

KeyboardInterrupt: 

In [ ]:
import matplotlib as mpl


In [227]:
import pickle
from glob import iglob
import numpy as np
import librosa
from shutil import rmtree
from constants import *
from constants import DATA_AUDIO_DIR
DATA_AUDIO_VAL_DIR = './test_val_data'
OUTPUT_DIR_VAL_TRAIN = './output_val/train'
OUTPUT_DIR_VAL_TEST = './output_val/test'

DATA_AUDIO_DIR = './test_train_data'
list_dir = os.listdir(DATA_AUDIO_DIR)
list_dir.sort()

def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


# def del_folder(path):
#     try:
#         rmtree(path)
#     except:
#         pass


# del_folder(OUTPUT_DIR_TRAIN)
# del_folder(OUTPUT_DIR_TEST)
# mkdir_p(OUTPUT_DIR_TRAIN)
# mkdir_p(OUTPUT_DIR_TEST)

class_id = {list_dir[i]: i for i in range(len(list_dir))}


def extract_class_id(wav_filename):
    return class_ids.get(wav_filename[16:-6])


def read_audio_from_filename(filename, target_sr):
    audio, _ = librosa.load(filename, sr=target_sr, mono=True)
    audio = audio.reshape(-1, 1)
    return audio


def convert_data():
    DATA_AUDIO_DIR = './test_train_data'
    list_dir = os.listdir(DATA_AUDIO_VAL_DIR)
    list_dir.sort()
    print(list_dir)
#     AUDIO_LENGTH = 200000
#     TARGET_SR = 16000
    for i, wav_filename in enumerate(iglob(os.path.join(DATA_AUDIO_VAL_DIR, '**/**.wav'), recursive=True)):
        print(i, wav_filename)
        class_id = extract_class_id(wav_filename)
        audio_buf = read_audio_from_filename(wav_filename, target_sr=TARGET_SR)
        # normalize mean 0, variance 1
        audio_buf = (audio_buf - np.mean(audio_buf)) / np.std(audio_buf)
        original_length = len(audio_buf)
        print(i, wav_filename, original_length, np.round(np.mean(audio_buf), 4), np.std(audio_buf))
        if original_length < AUDIO_LENGTH:
            audio_buf = np.concatenate((audio_buf, np.zeros(shape=(AUDIO_LENGTH - original_length, 1))))
            print('PAD New length =', len(audio_buf))
        elif original_length > AUDIO_LENGTH:
            audio_buf = audio_buf[0:AUDIO_LENGTH]
#             audio_buf = audio_buf[10000:AUDIO_LENGTH+10000]
            print('CUT New length =', len(audio_buf))

        output_folder = OUTPUT_DIR_VAL_TRAIN
#         if i % 5 == 0:
#             output_folder = OUTPUT_DIR_VAL_TEST

        output_filename = os.path.join(output_folder, str(wav_filename[16:-4]).replace('/', '_') + '.pkl')

        out = {'class_id': class_id,
               'audio': audio_buf,
               'sr': TARGET_SR}
        print("-----------------")
        print(out)
        with open(output_filename, 'wb') as w:
            pickle.dump(out, w)


if __name__ == '__main__':
    convert_data()

['2017019740001_kwakjuheon', '2017019740029_limdonggeun', '2017019770001_kwonyuna', '2017019880001_kimsubin']
0 ./test_val_data/2017019770001_kwonyuna/5.wav
0 ./test_val_data/2017019770001_kwonyuna/5.wav 187894 0.0 1.0
CUT New length = 20000
-----------------
{'class_id': 34, 'audio': array([[ 3.5577218e-04],
       [ 7.3198498e-05],
       [ 8.0742303e-04],
       ...,
       [-2.4037506e-01],
       [-2.8659305e-01],
       [ 4.7059527e-01]], dtype=float32), 'sr': 8000}
1 ./test_val_data/2017019770001_kwonyuna/1.wav
1 ./test_val_data/2017019770001_kwonyuna/1.wav 210080 -0.0 0.99999994
CUT New length = 20000
-----------------
{'class_id': 34, 'audio': array([[ 3.6729407e-04],
       [ 8.3726802e-05],
       [ 4.1892932e-04],
       ...,
       [-8.4960204e-01],
       [-3.2632080e-01],
       [ 7.6895215e-02]], dtype=float32), 'sr': 8000}
2 ./test_val_data/2017019770001_kwonyuna/3.wav
2 ./test_val_data/2017019770001_kwonyuna/3.wav 201206 0.0 1.0000001
CUT New length = 20000
----------

In [228]:
from sklearn.metrics import accuracy_score

train_files_1 = glob(os.path.join(OUTPUT_DIR_VAL_TRAIN, '**.pkl'))
train_files_1.sort()
# print(train_files_1)

x_tr_1, y_tr_1 = get_data(train_files_1)
# print('x_tr_1 : ', x_tr_1)
print('y_tr_1 : ', y_tr_1)

y_tr_12 = to_categorical(y_tr_1, num_classes=num_classes)
# print('y_tr_1 : ', y_tr_1)

pred_out = model.predict(x_tr_1)
# print('pred out : ', pred_out)

real =[]
pred_out_idex=[]

val_class_names = os.listdir(DATA_AUDIO_VAL_DIR)
print(val_class_names)
# val_class_names.sort()

DATA_AUDIO_DIR = './test_train_data'
list_dir = os.listdir(DATA_AUDIO_DIR)
list_dir.sort()

j = 0
for i,pred in zip(range(0, len(pred_out)), pred_out) :
#     print('i : ', i)
#     print('pred : ', pred)
    print("pre : ", np.argmax(pred))
    if i == 5:
        j += 1
    print("Predict :",list_dir[np.argmax(pred)], ", Real :", list_dir[y_tr_1[i]])
    real.append(list_dir.index(list_dir[y_tr_1[i]]))
    pred_out_idex.append(np.argmax(pred))
    
# get the accuracy
print (accuracy_score(real, pred_out_idex))

y_tr_1 :  [ 0  0  0  0  0 28 34 34 34 34 34 74 74 74 74 74]
['2017019770001_kwonyuna', '2017019740001_kwakjuheon', '2017019880001_kimsubin', '2017019740029_limdonggeun']
pre :  0
Predict : 2017019740001_kwakjuheon , Real : 2017019740001_kwakjuheon
pre :  0
Predict : 2017019740001_kwakjuheon , Real : 2017019740001_kwakjuheon
pre :  0
Predict : 2017019740001_kwakjuheon , Real : 2017019740001_kwakjuheon
pre :  0
Predict : 2017019740001_kwakjuheon , Real : 2017019740001_kwakjuheon
pre :  0
Predict : 2017019740001_kwakjuheon , Real : 2017019740001_kwakjuheon
pre :  28
Predict : 2017019740029_limdonggeun , Real : 2017019740029_limdonggeun
pre :  34
Predict : 2017019770001_kwonyuna , Real : 2017019770001_kwonyuna
pre :  34
Predict : 2017019770001_kwonyuna , Real : 2017019770001_kwonyuna
pre :  34
Predict : 2017019770001_kwonyuna , Real : 2017019770001_kwonyuna
pre :  34
Predict : 2017019770001_kwonyuna , Real : 2017019770001_kwonyuna
pre :  34
Predict : 2017019770001_kwonyuna , Real : 2017019